In [1]:
import numpy as np
import pandas as pd
# import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

2024-11-18 01:57:41.005645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-18 01:57:41.340447: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 01:57:42.234713: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-11-18 01:57:42.234812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
df = pd.read_csv("raw_gold_price.csv")
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
print(len(df))
df.head()

1718


,Date,Open,High,Low,Close,Adj Close,Volume,SP_open,SP_high,SP_low,...,GDX_Low,GDX_Close,GDX_Adj Close,GDX_Volume,USO_Open,USO_High,USO_Low,USO_Close,USO_Adj Close,USO_Volume
0,2011-12-15,154.740005,154.949997,151.710007,152.330002,152.330002,21521900,123.029999,123.199997,121.989998,...,51.570000,51.680000,48.973877,20605600,36.900002,36.939999,36.049999,36.130001,36.130001,12616700
1,2011-12-16,154.309998,155.369995,153.899994,155.229996,155.229996,18124300,122.230003,122.949997,121.300003,...,52.040001,52.680000,49.921513,16285400,36.180000,36.500000,35.730000,36.270000,36.270000,12578800
2,2011-12-19,155.479996,155.860001,154.360001,154.869995,154.869995,12547200,122.059998,122.320000,120.029999,...,51.029999,51.169998,48.490578,15120200,36.389999,36.450001,35.930000,36.200001,36.200001,7418200
3,2011-12-20,156.820007,157.429993,156.580002,156.979996,156.979996,9136300,122.180000,124.139999,120.370003,...,52.369999,52.990002,50.215282,11644900,37.299999,37.610001,37.220001,37.560001,37.560001,10041600
4,2011-12-21,156.979996,157.529999,156.130005,157.160004,157.160004,11996100,123.930000,124.360001,122.750000,...,52.419998,52.959999,50.186852,8724300,37.669998,38.240002,37.520000,38.110001,38.110001,10728000


In [3]:
news_df = pd.read_csv("news.csv")
news_df['Date'] = pd.to_datetime(news_df['Date'], format='%b %d, %Y')
news_df = news_df.groupby('Date')['Headline'].apply(' '.join).reset_index()
merged_df = pd.merge(df[['Date','Close','GDX_Close','SF_Price', 'EG_close', 'PLT_Price','USDI_Price','OF_Price', 'SP_close']], news_df, on='Date', how='outer')
merged_df = merged_df[merged_df['Date'] >= '2011-12-12']
combined_headline = ''
for index, row in merged_df.iterrows():
    if pd.isna(row['Close']):
        combined_headline+=row['Headline']
    else:
        if type(row['Headline'])==str:
            combined_headline+=row['Headline']
        merged_df.at[index,'Headline'] = combined_headline
        combined_headline=''
merged_df = merged_df.dropna(subset=['Close'])
merged_df = merged_df[merged_df['Headline'] != '']
print(len(merged_df))
merged_df.head()

954


,Date,Close,GDX_Close,SF_Price,EG_close,PLT_Price,USDI_Price,OF_Price,SP_close,Headline
30,2012-01-05,157.779999,53.900002,52740.0,73.199997,1421.25,81.250,112.74,128.039993,Wall Street Experts: UBS Analysts Love These 1...
34,2012-01-11,159.669998,54.310001,52607.0,71.699997,1499.55,81.614,112.24,129.199997,A. Schulman Enters Oversold Territory - Tale o...
35,2012-01-12,160.380005,54.720001,52770.0,70.650002,1498.95,81.036,111.26,129.509995,Zacks Releases Four Powerful ''Buy'' Stocks: I...
42,2012-01-24,162.009995,51.820000,55365.0,66.599998,1550.75,80.026,110.03,131.460007,Waters Outperforms Q4 Estimate - Analyst Blog
59,2012-02-17,167.350006,54.150002,55926.0,67.750000,1635.60,79.456,119.58,136.410004,Agilent Reports In Line - Analyst Blog


In [4]:
map_df = pd.read_csv("processed_gold_price.csv")[['Date','Close']]
map_df['Date'] = pd.to_datetime(map_df['Date'], format='%Y-%m-%d')
map_df['MAP_3'] = map_df['Close'].rolling(window=3).mean()
map_df['MAP_9'] = map_df['Close'].rolling(window=9).mean()
map_df.dropna(inplace=True)
map_df = map_df.merge(merged_df[['Date','Headline']], on='Date',how='inner')
print(len(map_df))
map_df.head(5)

954


,Date,Close,MAP_3,MAP_9,Headline
0,2012-01-05,157.779999,156.803335,153.959445,Wall Street Experts: UBS Analysts Love These 1...
1,2012-01-11,159.669998,158.269999,157.346666,A. Schulman Enters Oversold Territory - Tale o...
2,2012-01-12,160.380005,159.563334,157.842222,Zacks Releases Four Powerful ''Buy'' Stocks: I...
3,2012-01-24,162.009995,162.655557,161.775558,Waters Outperforms Q4 Estimate - Analyst Blog
4,2012-02-17,167.350006,167.820002,167.544444,Agilent Reports In Line - Analyst Blog


## Linear Regression

In [5]:
X = map_df.drop(columns=['Headline','Close','Date'])
Y = map_df['Close']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)
print("Percentage error:", np.mean(np.abs(Y_test-Y_pred/Y_test))/100)

Mean Squared Error: 0.41218101634604537
R^2 Score: 0.9980167281730175
Percentage error: 1.2212318890938203


## LSTM

In [6]:
numerical_features = map_df[map_df.columns[2:4]].values
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

# Preprocessing Text Features for LSTM
MAX_WORDS = 10000  # Maximum number of words in vocabulary
MAX_SEQ_LENGTH = 1000  # Maximum sequence length for padding

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(map_df['Headline'])

# Convert text to sequences and pad them
text_sequences = tokenizer.texts_to_sequences(map_df['Headline'])
text_features = pad_sequences(text_sequences, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")

# Labels
labels = map_df['Close'].values

# Train-Test Split
X_train_num, X_test_num, X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    numerical_features_scaled, text_features, labels, test_size=0.1, random_state=42
)

# Define Model
# Numerical Input
numerical_input = Input(shape=(2,), name="numerical_input")
X_num = Dense(8, activation="relu")(numerical_input)
X_num = Dropout(0.3)(X_num)

# Text Input with LSTM
text_input = Input(shape=(MAX_SEQ_LENGTH,), name="text_input")
embedding = Embedding(input_dim=MAX_WORDS, output_dim=32)(text_input)
X_text = LSTM(80, return_sequences=False)(embedding)
X_text = Dropout(0.2)(X_text)

# Combine Numerical and Text Branches
combined = concatenate([X_num, X_text])
X = Dense(200, activation="relu")(combined)
X = Dense(50, activation="relu")(combined)
X = Dropout(0.2)(X)
output = Dense(1, activation="linear")(X)

# Compile Model
model = Model(inputs=[numerical_input, text_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mse"])

# Train Model
history = model.fit(
    [X_train_num, X_train_text],
    Y_train,
    validation_split=0.1,
    epochs=35,
    batch_size=32,
    verbose=1
)

# Evaluate Model
Y_pred = model.predict([X_test_num, X_test_text])
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error on Test Set: {mse:.4f}")
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)
print("Percentage error:", np.mean(np.abs(Y_pred-Y_test/Y_test))/100)
print("Actual Price:",Y_test[:5])
print("Predicted_Price:",Y_pred[:5])

#Make new prediction
row = np.array(map_df)[50:60,1:]
X_num = row[:,1:3]
X_text = row[:,3]
Y = row[:,1]

new_numerical_features_scaled = scaler.transform(X_num)
new_text_sequence = tokenizer.texts_to_sequences(X_text)
new_text_features = pad_sequences(new_text_sequence, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
predicted_close_price = model.predict([new_numerical_features_scaled, new_text_features])

print("Actual Price:", Y)
print("Predicted Price:",end='')
for i, prediction in enumerate(predicted_close_price):
    print(prediction[0], end=' ')

2024-11-18 01:57:46.552990: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-18 01:57:46.553023: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: aditechbuddy-IdeaPad-Gaming-3-15ACH6
2024-11-18 01:57:46.553032: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: aditechbuddy-IdeaPad-Gaming-3-15ACH6
2024-11-18 01:57:46.553135: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.183.1
2024-11-18 01:57:46.553156: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.183.1
2024-11-18 01:57:46.553161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.183.1
2024-11-18 01:57:46.553395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to us

Epoch 1/35
25/25 [==============================] - 7s 226ms/step - loss: 14768.7100 - mse: 14768.7100 - val_loss: 13433.9844 - val_mse: 13433.9844
Epoch 2/35
25/25 [==============================] - 5s 208ms/step - loss: 11982.2236 - mse: 11982.2236 - val_loss: 10297.8057 - val_mse: 10297.8057
Epoch 3/35
25/25 [==============================] - 5s 205ms/step - loss: 8835.1201 - mse: 8835.1201 - val_loss: 7194.7964 - val_mse: 7194.7964
Epoch 4/35
25/25 [==============================] - 5s 202ms/step - loss: 5891.6265 - mse: 5891.6265 - val_loss: 4378.9702 - val_mse: 4378.9702
Epoch 5/35
25/25 [==============================] - 5s 202ms/step - loss: 3378.5488 - mse: 3378.5488 - val_loss: 2177.5676 - val_mse: 2177.5676
Epoch 6/35
25/25 [==============================] - 5s 209ms/step - loss: 1626.5314 - mse: 1626.5314 - val_loss: 822.4566 - val_mse: 822.4566
Epoch 7/35
25/25 [==============================] - 5s 210ms/step - loss: 668.4285 - mse: 668.4285 - val_loss: 296.3397 - val_mse:

In [9]:
import pickle
with open('model_map_headlines.pkl', 'wb') as f:
    pickle.dump(model, f)

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer,f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

INFO:tensorflow:Assets written to: ram://b95835f9-f52a-4c00-8b38-553da26c9016/assets


INFO:tensorflow:Assets written to: ram://b95835f9-f52a-4c00-8b38-553da26c9016/assets


## Sentiment Analysis

In [10]:
sent_df = map_df.copy()
sent_df['Price_Diff'] = sent_df['Close'].diff()
sent_df.drop(index=sent_df.index[0], inplace=True)
sent_df.drop(['Date'], axis=1, inplace=True)
sent_df.head()

,Close,MAP_3,MAP_9,Headline,Price_Diff
1,159.669998,158.269999,157.346666,A. Schulman Enters Oversold Territory - Tale o...,1.889999
2,160.380005,159.563334,157.842222,Zacks Releases Four Powerful ''Buy'' Stocks: I...,0.710007
3,162.009995,162.655557,161.775558,Waters Outperforms Q4 Estimate - Analyst Blog,1.629990
4,167.350006,167.820002,167.544444,Agilent Reports In Line - Analyst Blog,5.340011
5,166.610001,168.130000,170.912221,Waters Reiterates at Neutral - Analyst Blog,-0.740005


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load FinBERT model and tokenizer
model_name = "yiyanghkust/finbert-tone"  # Pretrained FinBERT model for financial sentiment
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to compute sentiment score
def compute_sentiment_score(texts):
    """
    Compute sentiment score for a list of financial texts using FinBERT.
    :param texts: List of strings containing financial news or headlines.
    :return: List of sentiment scores in the range (-1, 1).
    """
    scores = []
    for text in texts:
        # Tokenize the input text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        # Perform inference
        with torch.no_grad():
            outputs = model(**inputs)
        # Get probabilities for each sentiment class
        logits = outputs.logits.squeeze().tolist()
        # Calculate sentiment score: positive logit - negative logit
        score = logits[2] - logits[0]
        scores.append(score)
    return np.array(scores)

# Assuming `news_df` has a column 'headline' containing the financial news headlines
scores = compute_sentiment_score(sent_df["Headline"].tolist())
sent_df['Sentiment_Scores'] = (scores-np.mean(scores))/np.std(scores)

# Display the updated DataFrame
print(sent_df.head())

ModuleNotFoundError: No module named 'torch'

In [ ]:
X = sent_df.drop(columns=['Close','Headline','Price_Diff'])
Y = sent_df['Close']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)
print("Percentage error:", np.mean(np.abs(Y_test-Y_pred/Y_test))/100)

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# user_input = request.POST.get('user_input')
user_input="Gold Hits Record High as Analysts Predict Continued Strength Amid Economic Uncertainty"
model_name = "yiyanghkust/finbert-tone"  # Pretrained FinBERT model for financial sentiment
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize the input text
inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=512)

with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits.squeeze().tolist()

score = logits[2] - logits[0]

print(score)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

2.1534371376037598
